#**Building function to build model:**


*   Resnet Block
*   Densnet Block
*  Upsampling Block
* Convolution Block
* Generator
* Discriminator



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Add, Lambda, LeakyReLU, Flatten, Dense,PReLU,Concatenate
from tensorflow.keras.layers import PReLU
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt

Convolution Block for generator include 

1.   Conv2D
2.    Activation layer (PRelu, leakyRelu)



In [ ]:
def convBlock(inputs,num_ker=64,ker_size=3,active='prelu',strides=1,name='conv'):
   x=Conv2D(filters=num_ker,kernel_size=ker_size,padding='same',strides=strides,name=name)(inputs)
   if active=='prelu':
      x=PReLU(alpha_initializer=tf.constant_initializer(0.2),shared_axes=(1,2),name='PRelu_'+name)(x)
   elif active=='leakyrelu':
      x=LeakyReLU(alpha=0.2,name='leakyRelu_'+name)(x)
   return x   
    

DenseBlock include number (no_conv_block) of ConvBlock with Concatenate layer to implement Dense Architecture

In [ ]:
#Residual in residual Dense Block
def DenseBlock(inputs,no_conv_block=5,no_filter=64,ker_size=3,name='denseBlock'):
  x=convBlock(inputs=inputs,num_ker=no_filter,ker_size=ker_size,active='prelu',name='conv.0_'+name)
  for i in range(no_conv_block-1):
    x=Concatenate(axis=-1)([inputs,x])
    if(i<no_conv_block-2):
      inputs=convBlock(inputs=x,num_ker=no_filter,ker_size=ker_size,active='prelu',name='conv.'+str(i+1)+'_'+name)
    else:
      inputs=convBlock(inputs=x,num_ker=no_filter,ker_size=ker_size,active='none',name='conv.'+str(i+1)+'_'+name)
  return inputs

ResnetBlock include number (no_Block) of denseBlock with Add() layer to implement Resnet Architecture, this block also called "RDBB" (Residual in residual dense block)

In [ ]:
def ResnetBlock(inputs,no_Block=3,beta=0.2,name='resnetBlock',no_conv_block=5,no_filter=64):
  x=DenseBlock(inputs,no_filter=no_filter,no_conv_block=no_conv_block,name='denseBlock.0_'+name)
  x=Add()([inputs,x*beta])
  for i in range(no_Block-1):
    x=DenseBlock(x,no_filter=no_filter,no_conv_block=no_conv_block,name='denseBlock.'+str(i+1)+'_'+name)
    x=Add()([inputs,x*beta])
  return Add()([inputs,x*beta])

UpsamplingBlock is implemented of Sub-pixel convolution (Shuffle-pixel) to upsampling fearture size

In [ ]:
def upsamplingBlock(inputs,input_dimens=64,scale=2,name='shuffle'):
  x=convBlock(inputs,num_ker=input_dimens*(scale**2),ker_size=3,active='none',name='conv.0_'+name)
  x=tf.nn.depth_to_space(x, scale)
  x=PReLU(alpha_initializer=tf.constant_initializer(0.2),shared_axes=(1,2),name='PRelu.0_'+name)(x)
  return x

discriminatorConvBlock include convBlock for Discriminator model

In [ ]:
def discriminatorConvBlock(inputs,ker_size=3,num_ker=64,with_bn=True,name='convblock'):
  x=convBlock(inputs,ker_size=ker_size,num_ker=num_ker,active='leakyrelu',strides=1,name=name)
  if with_bn:
    x=BatchNormalization(momentum=0.8)(x)
  x=convBlock(x,ker_size=ker_size,num_ker=num_ker,active='leakyrelu',strides=2,name='strided_'+name)
  return x

Building Generator

In [ ]:
def build_generator(no_dense_per_res=3,no_resblock=8,no_dense=5,kn_construc=3,no_filter=64):
  inputs = keras.Input(shape=(None,None, 3), name="img")
  x=x_0=convBlock(inputs,num_ker=no_filter,ker_size=kn_construc,active='prelu',name='conv_extraction')

  x=ResnetBlock(x,no_filter=no_filter,no_Block=no_dense_per_res,no_conv_block=no_dense,name='resnetBlock.0')
  for i in range(no_resblock-1):
    x=ResnetBlock(x,no_filter=no_filter,no_Block=no_dense_per_res,no_conv_block=no_dense,name='resnetBlock.'+str(i+1))

  scale=2
  x=convBlock(x,num_ker=no_filter,ker_size=3,active='none',name='conv_mapping')
  x=Add()([x_0,x])
  
  for i in range(scale):
    x= upsamplingBlock(x,scale=2,input_dimens=no_filter,name='shuffle.'+str(i))
  x=convBlock(x,num_ker=no_filter,ker_size=3,active='prelu',name='mapping_after_upsample')

  x=convBlock(x,num_ker=3,ker_size=kn_construc,active='none',name='conv_recontructed')
  model=keras.Model(inputs,x,name='generator')
  model.summary()
  return model

Building Discriminator

In [ ]:
def build_discriminator(input_shape=(128,128,3),no_block=4):
  inputs=tf.keras.Input(shape=input_shape,name='img')
  num_ker=64
  x=inputs
  # x=tf.keras.layers.RandomCrop(128, 128,name='crop')(inputs)
  for i in range(no_block):
    x=discriminatorConvBlock(x,ker_size=3,num_ker=num_ker,with_bn=False,name='conv.'+str(i))
    num_ker=num_ker*2
  x=Flatten()(x)
  x=Dense(1024)(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = Dense(1)(x)
  discriminator=keras.Model(inputs=inputs,outputs=x,name='discriminator')
  discriminator.summary()
  return discriminator

In [ ]:
# model1=build_generator(no_dense=5,no_dense_per_res=3,no_resblock=5,kn_construc=5)
# model2=build_generator_separated(no_dense=5,no_dense_per_res=3,no_resblock=5,kn_construc=5)

In [ ]:
# keras.utils.plot_model(model1, "generator.png", show_shapes=True, expand_nested=True)
# keras.utils.plot_model(model2, "generator2.png", show_shapes=True, expand_nested=True)

In [ ]:
# build_discriminator()

##**Building Loss Function include:**


*   GAN Loss
*   Perceptual Loss



In [ ]:
def loss_compute(type='l1'):
    """pixel loss"""
    if type == 'l1':
        return tf.keras.losses.MeanAbsoluteError()
    else:
        return tf.keras.losses.MeanSquaredError()


In [ ]:
def featuremap_vgg19(index=5,with_activation=False):
  #vgg19_54, conv 4th before activation, after 5th maxPooling (20 th layers)
  #vgg19_22, conv 2th befor activation, after 2th maxpooling (5th layers)
  #for style we use 5th, 10th, 15th, 20th layers 
    vgg19 = tf.keras.applications.VGG19(include_top=False,input_shape=(None,None,3), weights='imagenet')
    if with_activation==False:
      vgg19.layers[index].activation=None
    perceptual_model=tf.keras.Model(inputs=vgg19.input,outputs=vgg19.layers[index].output)
    return perceptual_model

In [ ]:
def style_loss(type_loss='l2', index_layer=5, with_activation=False):
    """style_loss"""
    loss_function=loss_compute(type=type_loss)
    extract_model=featuremap_vgg19(index=index_layer,with_activation=with_activation)
    def gram_matrix(input_tensor):
      result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
      input_shape = tf.shape(input_tensor)
      num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
      out_size=tf.cast(input_shape[1]*input_shape[2]*input_shape[3], tf.float32)
      return result/(num_locations),out_size
    @tf.function
    def content_loss(hr, sr):
        # the input scale range is [0, 1] (vgg is [0, 255]).
        # 12.75 is rescale factor for vgg featuremaps.
        preprocess_sr = tf.keras.applications.vgg19.preprocess_input(sr * 255.) 
        preprocess_hr = tf.keras.applications.vgg19.preprocess_input(hr * 255.) 
        sr_features,size = gram_matrix(extract_model(preprocess_sr))
        hr_features,size = gram_matrix(extract_model(preprocess_hr))
        return tf.reduce_sum(tf.square(sr_features-hr_features))/(4*size**2)
    return content_loss

In [ ]:
def feature_loss(type_loss='l2', index_layer=5, with_activation=False):
    """content loss"""
    loss_function=loss_compute(type=type_loss)
    extract_model=featuremap_vgg19(index=index_layer,with_activation=with_activation)
    @tf.function
    def content_loss(hr, sr):
        # the input scale range is [0, 1] (vgg is [0, 255]).
        # 12.75 is rescale factor for vgg featuremaps.
        preprocess_sr = tf.keras.applications.vgg19.preprocess_input(sr * 255.) 
        preprocess_hr = tf.keras.applications.vgg19.preprocess_input(hr * 255.) 
        sr_features = extract_model(preprocess_sr)
        hr_features = extract_model(preprocess_hr) 

        return  loss_function(sr_features,hr_features)

    return content_loss


In [ ]:
def build_featureloss(low_index=5,high_index=20,type_loss='l2',alpha1=1,alpha2=1):
  low_feature=feature_loss(type_loss=type_loss,index_layer=low_index)
  high_feature=feature_loss(type_loss=type_loss,index_layer=high_index)
  @tf.function
  def content_loss(sr, hr):
    return tf.reduce_sum([alpha1*low_feature(sr,hr),alpha2*high_feature(sr,hr)])
  return content_loss

In [ ]:
def build_styleloss(type_loss='l2',list_alpha=[0.25,0.25,0.25,0.25]):
  index_layer=[5,10,15,20]
  list_feature=[style_loss(type_loss=type_loss,index_layer=i) for i in index_layer]
  if(len(list_alpha)!=4):
     raise Exception("alphas length out of range 4")
  @tf.function
  def content_loss(sr,hr):
    return tf.reduce_sum([list_alpha[index]*style(sr,hr) for index,style in enumerate(list_feature)])
  return content_loss

In [ ]:
def psnr_metrics(max_val=1):
  @tf.function
  def psnr(sr,hr):
    return tf.reduce_mean(tf.image.psnr(sr,hr,max_val=1))
  return psnr

In [ ]:
def ssim_metrics(max_val=1):
  @tf.function
  def ssim(sr,hr):
    return tf.reduce_mean(tf.image.ssim(sr, hr, max_val=max_val, filter_size=11, filter_sigma=1.5, k1=0.01, k2=0.03))
  return ssim

#Adversarial loss

In [ ]:
#type loss with = 'gan','lsgan','wgan','hingegan'
def discriminator_loss(type_loss='gan',with_relativistic=True):
  if with_relativistic==True:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        sr_deal=(sr_logits - tf.reduce_mean(hr_logits))
        hr_deal=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.square(hr_deal - 1.0))
        loss_sr=tf.reduce_mean(tf.square(sr_deal + 1.0))
        return loss_hr +loss_sr
      return lsgan_loss
    elif type_loss=='hinge':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        sr_deal=(sr_logits - tf.reduce_mean(hr_logits))
        hr_deal=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.nn.relu(1.0 - hr_deal))
        loss_sr=tf.reduce_mean(tf.nn.relu(1.0 + sr_deal))
        return loss_hr +loss_sr
      return hingegan_loss
    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        sr_deal=(sr_logits - tf.reduce_mean(hr_logits))
        hr_deal=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(hr_deal), logits=hr_deal))
        loss_sr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(sr_deal), logits=sr_deal))
        return loss_hr +loss_sr
      return standard_loss
  ## without Relativistic Average
  else:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        loss_hr=tf.reduce_mean(tf.square(hr_logits - 1.0))
        loss_sr=tf.reduce_mean(tf.square(sr_logits ))
        return loss_hr +loss_sr
      return lsgan_loss

    elif type_loss=='hinge':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        loss_hr=tf.reduce_mean(tf.nn.relu(1.0 - hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.relu(1.0 + sr_logits ))
        return loss_hr +loss_sr
      return hingegan_loss

    elif type_loss=='wgan':
      @tf.function
      def wgan_loss(sr_logits,hr_logits):
        loss_hr=-tf.reduce_mean(hr_logits)
        loss_sr=tf.reduce_mean(sr_logits)
        return loss_hr +loss_sr
      return wgan_loss
    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        loss_hr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(hr_logits), logits=hr_logits))
        loss_sr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(sr_logits), logits=sr_logits))
        return loss_hr +loss_sr
      return standard_loss

In [ ]:
#type loss with = 'gan','lsgan','wgan','hingegan'
def generator_loss(type_loss='gan',with_relativistic=True):
  if with_relativistic==True:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        sr_deal=(sr_logits - tf.reduce_mean(hr_logits))
        hr_deal=(hr_logits - tf.reduce_mean(sr_logits))

        loss_hr=tf.reduce_mean(tf.square(hr_deal + 1.0))
        # hr --> -1.0
        loss_sr=tf.reduce_mean(tf.square(sr_deal - 1.0))
        # sr --> 1.0
        return loss_hr +loss_sr
      return lsgan_loss

    elif type_loss=='hinge':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        sr_deal=(sr_logits - tf.reduce_mean(hr_logits))
        hr_deal=(hr_logits - tf.reduce_mean(sr_logits))

        loss_hr=tf.reduce_mean(tf.nn.relu(1.0 + hr_deal))
        # hr --> -1.0
        loss_sr=tf.reduce_mean(tf.nn.relu(1.0 - sr_deal))
        # sr --> 1.0
        return loss_hr +loss_sr
      return hingegan_loss

    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        sr_deal=(sr_logits - tf.reduce_mean(hr_logits))
        hr_deal=(hr_logits - tf.reduce_mean(sr_logits))
        loss_hr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(hr_deal), logits=hr_deal))
        loss_sr=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(sr_deal), logits=sr_deal))
        return loss_hr +loss_sr
      return standard_loss
  ## without Relativistic Average
  else:
    if type_loss=='lsgan':
      @tf.function
      def lsgan_loss(sr_logits,hr_logits):
        return tf.reduce_mean(tf.square(sr_logits - 1.0))
      return lsgan_loss
    elif type_loss=='hinge' or type_loss=='wgan':
      @tf.function
      def hingegan_loss(sr_logits,hr_logits):
        return -tf.reduce_mean(sr_logits)
      return hingegan_loss
    else:
      @tf.function
      def standard_loss(sr_logits,hr_logits):
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(sr_logits), logits=sr_logits))
      return standard_loss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! pwd
%cd drive/MyDrive
%cd Colab-Super_resolution/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
/content/drive/MyDrive
/content/drive/MyDrive/Colab-Super_resolution


In [ ]:
!pip install import-ipynb
import import_ipynb
import tensorflow as tf
import LoadingDataset  as dataset_loader

importing Jupyter notebook from LoadingDataset.ipynb
import libary...


In [ ]:
import tensorflow_datasets as tfds
(train_ds, validate), metadata = tfds.load(
    'div2k/bicubic_x4',
    split=['train','validation'],
    with_info=True,
    as_supervised=True,
   )

In [ ]:
cropped_size = 256
batch_size = 8
# train = train.map(lambda lr,hr:(dataset_loader.random_crop(lr,hr,cropped_size,4)), num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train = train.map(lambda lr,hr:( dataset_loader.normalize_img(lr),dataset_loader.normalize_img(hr)), num_parallel_calls=tf.data.AUTOTUNE)
train = train.map(dataset_loader.augment,num_parallel_calls=tf.data.AUTOTUNE)
train = dataset_loader.configure_performan(train,BUFFER_SIZE=800,BATCH_SIZE=batch_size)

In [ ]:
validate = validate.map(lambda lr,hr:(dataset_loader.random_crop(lr,hr,cropped_size,4)), num_parallel_calls=tf.data.AUTOTUNE)
validate = validate.map(lambda lr,hr:( dataset_loader.normalize_img(lr),dataset_loader.normalize_img(hr)), num_parallel_calls=tf.data.AUTOTUNE)
# train = train.map(dataset_loader.augment,num_parallel_calls=tf.data.AUTOTUNE)
validate = dataset_loader.configure_performan(validate,BUFFER_SIZE=100,BATCH_SIZE=batch_size)

Normalizes images: `uint8` -> `float32`
Normalizes images: `uint8` -> `float32`


In [ ]:
low=[]
high=[]
style=[]
for i in validate.take(len(validate)):
  for j in range(len(i[0])):
    low.append(feature_loss_1(tf.expand_dims(generator_image(i[0][j]),axis=0),tf.expand_dims(i[1][j],axis=0)))
    high.append(feature_loss_2(tf.expand_dims(generator_image(i[0][j]),axis=0),tf.expand_dims(i[1][j],axis=0)))
    style.append(st_loss(tf.expand_dims(generator_image(i[0][j]),axis=0),tf.expand_dims(i[1][j],axis=0)))

In [ ]:
tf.reduce_mean(low)


<tf.Tensor: shape=(), dtype=float32, numpy=423.49695>

In [ ]:
6e-3*tf.reduce_mean(high)

<tf.Tensor: shape=(), dtype=float32, numpy=438.7368>

In [ ]:
st={}
st['1']=[]
st['2']=[]
st['3']=[]
st['4']=[]
for i in style:
  st['1'].append(i[0])
  st['2'].append(i[1])
  st['3'].append(i[2])
  st['4'].append(i[3])

In [ ]:
st.values

<function dict.values>

In [ ]:
for i in st:
  print(tf.reduce_mean(st[str(i)]))

tf.Tensor(0.6872412, shape=(), dtype=float32)
tf.Tensor(527.7972, shape=(), dtype=float32)
tf.Tensor(1316.9547, shape=(), dtype=float32)
tf.Tensor(1.6663078, shape=(), dtype=float32)


In [ ]:
def generator_image(lr):
  return tf.image.resize(lr, [lr.shape[0]*4,lr.shape[1]*4], method=tf.image.ResizeMethod.BICUBIC)

In [ ]:
def random_crop(lr_img, hr_img, hr_crop_size, scale):
    lr_crop_size = hr_crop_size // scale
    lr_shape = tf.shape(lr_img)[:2]
    lr_top = tf.random.uniform(shape=(), maxval=lr_shape[0] - lr_crop_size + 1, dtype=tf.int32)
    lr_left = tf.random.uniform(shape=(), maxval=lr_shape[1] - lr_crop_size + 1, dtype=tf.int32)
    hr_top = lr_top * scale
    hr_left = lr_left * scale
    lr_crop = lr_img[lr_top:lr_top + lr_crop_size, lr_left:lr_left + lr_crop_size]
    hr_crop = hr_img[hr_top:hr_top + hr_crop_size, hr_left:hr_left + hr_crop_size]
    return lr_crop, hr_crop

In [ ]:
def build_styleloss_2(type_loss='l2',list_alpha=[0.25,0.25,0.25,0.25]):
  index_layer=[5,10,15,20]
  list_feature=[style_loss(type_loss=type_loss,index_layer=i) for i in index_layer]
  if(len(list_alpha)!=4):
     raise Exception("alphas length out of range 4")
  @tf.function
  def content_loss(sr,hr):
    return [list_alpha[index]*style(sr,hr) for index,style in enumerate(list_feature)]
  return content_loss

In [ ]:
st_loss=build_styleloss_2()

In [ ]:
feature_loss_1=build_featureloss(alpha1=0,alpha2=1)
feature_loss_2=build_featureloss(alpha1=1,alpha2=0)


#Operator

In [ ]:
def gram_matrix(input_tensor):
      result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
      print(result.shape)
      input_shape = tf.shape(input_tensor)
      print(input_shape)
      num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
      return result/(num_locations)

In [ ]:
def imshow(image, title=None):
  plt.figure(figsize=(10, 10))
  plt.imshow(image[0,:,:,:])

In [ ]:
def show_feature(input,number=8,title=None):
  # plot all 64 maps in an 8x8 squares
  square = number
  ix = 1
  plt.figure(figsize=(4*number, 4*number))
  for _ in range(square):
    for _ in range(square):
		# specify subplot and turn of axis
      ax = plt.subplot(square, square, ix)
      ax.set(title=ix)
      plt.imshow(input[0, :, :, ix-1], cmap='gray',aspect='equal')
      ix += 1
  plt.show()
# show the figure

In [ ]:
def gram_matrixa(A):
    channels = int(A.shape[-1])
    a = tf.reshape(A, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

#Metrics

In [ ]:
# !pip install tensorflow-addons

     |████████████████████████████████| 1.1 MB 22.5 MB/s 


In [ ]:
# import tensorflow_addons as tfa

In [ ]:
# content_path = tf.keras.utils.get_file('YellowLaasrLooking_new.jpg', 'https://www.sgv.edu.vn/uploads/images/info/city-state-la-gi.png')
# img = tf.keras.preprocessing.image.load_img(content_path)
# img=tf.keras.preprocessing.image.img_to_array(img,dtype='float32')
# img1=tfa.image.gaussian_filter2d(img) 

In [ ]:
# img2=tf.image.resize(img[0], [tf.round(img[0].shape[0]/4),tf.round(img[0].shape[1]/4)], method=tf.image.ResizeMethod.BICUBIC)

In [ ]:
# img2=tf.image.resize(img2, [img[0].shape[0],img[0].shape[1]], method=tf.image.ResizeMethod.BICUBIC)

In [ ]:
# img2=tf.expand_dims(img2,axis=0)